In [1]:
import os
import numpy as np
import pandas as pd
import warnings

# This is not recommended but I am doing this to suppress warnings from SARIMAX
warnings.simplefilter('ignore')

countryName = 'Italy'

nFeatures = 1

nDaysMin = 6
k = 5

nValid = 10
nTest = 10

dataDir = os.path.join('data', 'JHU', 'upto07082020_forPublication')


# confirmedFilename = 'time_series_covid19_confirmed_global.csv'
# deathsFilename = 'time_series_covid19_deaths_global.csv'
# recoveredFilename = 'time_series_covid19_recovered_global.csv'

confirmedFilename = 'https://raw.githubusercontent.com/arkobarman/covid-19_timeSeriesAnalysis/master/data/JHU/upto07082020_forPublication/time_series_covid19_confirmed_global.csv'
deathsFilename = 'https://raw.githubusercontent.com/arkobarman/covid-19_timeSeriesAnalysis/master/data/JHU/upto07082020_forPublication/time_series_covid19_deaths_global.csv'
recoveredFilename = 'https://raw.githubusercontent.com/arkobarman/covid-19_timeSeriesAnalysis/master/data/JHU/upto07082020_forPublication/time_series_covid19_recovered_global.csv'

In [2]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps, k):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix + k >= len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:end_ix+k]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [3]:
def meanAbsolutePercentageError(yTrueList, yPredList):
    absErrorList = [np.abs(yTrue - yPred) for yTrue, yPred in zip(yTrueList, yPredList)]
    absPcErrorList = [absError/yTrue for absError, yTrue in zip(absErrorList, yTrueList)]
    MAPE = 100*np.mean(absPcErrorList)
    return MAPE

def meanAbsolutePercentageError_kDay(yTrueListList, yPredListList):
    # Store true and predictions for day 1 in a list, day 2 in a list and so on
    # Keep each list of these lists in a respective dict with key as day #
    yTrueForDayK = {}
    yPredForDayK = {}
    for i in range(len(yTrueListList[0])):
        yTrueForDayK[i] = []
        yPredForDayK[i] = []
    for yTrueList, yPredList in zip(yTrueListList, yPredListList):
        for i in range(len(yTrueList)):
            yTrueForDayK[i].append(yTrueList[i])
            yPredForDayK[i].append(yPredList[i])
            
    # Get MAPE for each day in a list
    MAPEList = []
    for i in yTrueForDayK.keys():
        MAPEList.append(meanAbsolutePercentageError(yTrueForDayK[i], yPredForDayK[i]))
    return np.mean(MAPEList)

def meanForecastError(yTrueList, yPredList):
    forecastErrors = [yTrue - yPred for yTrue, yPred in zip(yTrueList, yPredList)]
    MFE = np.mean(forecastErrors)
    return MFE

def meanAbsoluteError(yTrueList, yPredList):
    absErrorList = [np.abs(yTrue - yPred) for yTrue, yPred in zip(yTrueList, yPredList)]
    return np.mean(absErrorList)

def meanSquaredError(yTrueList, yPredList):
    sqErrorList = [np.square(yTrue - yPred) for yTrue, yPred in zip(yTrueList, yPredList)]
    return np.mean(sqErrorList)

def rootMeanSquaredError(yTrueList, yPredList):
    return np.sqrt(meanSquaredError(yTrueList, yPredList))

def medianSymmetricAccuracy(yTrueList, yPredList):
    '''https://helda.helsinki.fi//bitstream/handle/10138/312261/2017SW001669.pdf?sequence=1'''
    logAccRatioList = [np.abs(np.log(yPred/yTrue)) for yTrue, yPred in zip(yTrueList, yPredList)]
    MdSA = 100*(np.exp(np.median(logAccRatioList))-1)
    return MdSA

def medianSymmetricAccuracy_kDay(yTrueListList, yPredListList):
    # Store true and predictions for day 1 in a list, day 2 in a list and so on
    # Keep each list of these lists in a respective dict with key as day #
    yTrueForDayK = {}
    yPredForDayK = {}
    for i in range(len(yTrueListList[0])):
        yTrueForDayK[i] = []
        yPredForDayK[i] = []
    for yTrueList, yPredList in zip(yTrueListList, yPredListList):
        for i in range(len(yTrueList)):
            yTrueForDayK[i].append(yTrueList[i])
            yPredForDayK[i].append(yPredList[i])
    # Get MdSA for each day in a list
    MdSAList = []
    for i in yTrueForDayK.keys():
        MdSAList.append(medianSymmetricAccuracy(yTrueForDayK[i], yPredForDayK[i]))
    return(np.mean(MdSAList))

In [4]:
# Function to get all three frames for a given country
def getCountryCovidFrDict(countryName):
    countryCovidFrDict = {}
    for key in covidFrDict.keys():
        dataFr = covidFrDict[key]
        countryCovidFrDict[key] = dataFr[dataFr['Country/Region'] == countryName]
    return countryCovidFrDict

In [5]:
# Load all 3 csv files
covidFrDict = {}
# covidFrDict['confirmed'] = pd.read_csv(os.path.join(dataDir, confirmedFilename))
# covidFrDict['deaths'] = pd.read_csv(os.path.join(dataDir, deathsFilename))
covidFrDict['confirmed'] = pd.read_csv(confirmedFilename)
covidFrDict['deaths'] = pd.read_csv(deathsFilename)

# Recovered is back again!
covidFrDict['recovered'] = pd.read_csv(recoveredFilename)

countryCovidFrDict = getCountryCovidFrDict(countryName)

# Get list of dates
colNamesList = list(countryCovidFrDict['confirmed'])
dateList = [colName for colName in colNamesList if '/20' in colName]
dataList = [countryCovidFrDict['confirmed'][date].iloc[0] for date in dateList]
dataDict = dict(zip(dateList, dataList))

# Get time series for cases > 100 only
daysSince = 100
nCasesGreaterDaysSinceList = []
datesGreaterDaysSinceList = []

for key in dataDict.keys():
    if dataDict[key] > daysSince:
        datesGreaterDaysSinceList.append(key)
        nCasesGreaterDaysSinceList.append(dataDict[key])
        
XList, yList = split_sequence(nCasesGreaterDaysSinceList, nDaysMin, k)

XTrainList = XList[0:len(XList)-(nValid + nTest)]
XValidList = XList[len(XList)-(nValid+nTest):len(XList)-(nTest)]
XTestList = XList[-nTest:]

yTrain = yList[0:len(XList)-(nValid + nTest)]
yValid = yList[len(XList)-(nValid+nTest):len(XList)-(nTest)]
yTest = yList[-nTest:]

print('Total size of data points for LSTM:', len(yList))
print('Size of training set:', len(yTrain))
print('Size of validation set:', len(yValid))
print('Size of test set:', len(yTest))

# Convert from list to matrix
XTrain = XTrainList.reshape((XTrainList.shape[0], XTrainList.shape[1], nFeatures))
XValid = XValidList.reshape((XValidList.shape[0], XValidList.shape[1], nFeatures))
XTest = XTestList.reshape((XTestList.shape[0], XTestList.shape[1], nFeatures))

Total size of data points for LSTM: 126
Size of training set: 106
Size of validation set: 10
Size of test set: 10


# Vanilla LSTM

In [6]:
# import tensorflow as tf
# from tensorflow.keras import Sequential
# from tensorflow.keras.layers import LSTM, Dense
# from tensorflow.keras.optimizers import Adam
# nNeurons = 100
# # define model
# model = Sequential()
# model.add(LSTM(nNeurons, activation='relu', input_shape=(nDaysMin, nFeatures)))
# model.add(Dense(1))
# opt = Adam(learning_rate=0.1)
# model.compile(optimizer=opt, loss='mse')

# XTestNew = XTest.copy()
# for day in range(k):
#     print('Day:', day)
#     yPred = model.predict(XTestNew, verbose=0)
#     for i in range(len(yPred)):
#         yPredListList[i].append(yPred[i][0])
#     print('Prediction:', yPred)
#     XTestNew = np.delete(XTestNew, 0, axis=1)
#     yPred = np.expand_dims(yPred, 2)
#     XTestNew = np.append(XTestNew, yPred, axis=1)

In [7]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

nNeurons = 100
nFeatures = 1

bestValidMAPE = 100
bestSeed = -1
for seed in range(100):
    tf.random.set_seed(seed=seed)
    
    # define model
    model = Sequential()
    model.add(LSTM(nNeurons, activation='relu', input_shape=(nDaysMin, nFeatures)))
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')

    # fit model
    model.fit(XTrain, yTrain[:,0], epochs=1000, verbose=0)

    yPredListList = []
    for day in range(nTest):
        yPredListList.append([])
    XValidNew = XValid.copy()
    for day in range(k):
        yPred = model.predict(XValidNew, verbose=0)
        for i in range(len(yPred)):
            yPredListList[i].append(yPred[i][0])
        XValidNew = np.delete(XValidNew, 0, axis=1)
        yPred = np.expand_dims(yPred, 2)
        XValidNew = np.append(XValidNew, yPred, axis=1)

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError_kDay(yValid, yPredListList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(LSTM(nNeurons, activation='relu', input_shape=(nDaysMin, nFeatures)))
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')

# fit model
model.fit(XTrain, yTrain[:,0], epochs=1000, verbose=0)

yPredListList = []
for day in range(nTest):
    yPredListList.append([])
XTestNew = XTest.copy()
for day in range(k):
    yPred = model.predict(XTestNew, verbose=0)
    for i in range(len(yPred)):
        yPredListList[i].append(yPred[i][0])
    XTestNew = np.delete(XTestNew, 0, axis=1)
    yPred = np.expand_dims(yPred, 2)
    XTestNew = np.append(XTestNew, yPred, axis=1)
    
MAPE = meanAbsolutePercentageError_kDay(yTest, yPredListList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy_kDay(yTest, yPredListList)
print('Test MdSA:', MdSA)

0 99.84852937452823
Updating best MAPE to 99.84852937452823...
Updating best seed to 0...
1 0.08540284468080865
Updating best MAPE to 0.08540284468080865...
Updating best seed to 1...
2 0.9727702950749055
3 1.7007722657214093
4 1.3184297104236748
5 0.849154107771583
6 2.8819542593371974
7 0.09241157031734129
8 0.17814643373845723
9 2.0438294235004335
10 0.5859250674753633
11 0.8347491455564129
12 0.11840838294300098
13 0.1025542358758309
14 99.85168320244462
15 2.0162151282595184
16 4.1366379656990855
17 0.21604907401302736
18 99.83412919956626
19 0.12098661206159209
20 2.2527128801360186
21 0.8166922199717342
22 99.83521576716812
23 0.14084655746204888
24 99.83040000803904
25 0.973768422843305
26 2.2891414391891756
27 99.84311137998903
28 99.94453282504416
29 0.2494216671778906
30 7.017034017683909
31 99.84749271169645
32 2.0708177979392293
33 0.11853383818586474
34 99.83581187324879
35 0.9498823641024552
36 1.4359239382308935
37 1.624668846457899
38 0.8179572200253891
39 5.1118102879

# Stacked LSTM

In [8]:
# define model
nNeurons = 50
nFeatures = 1

bestValidMAPE = 100
bestSeed = -1
for seed in range(100):
    tf.random.set_seed(seed=seed)
    model = Sequential()
    model.add(LSTM(nNeurons, activation='relu', return_sequences=True, input_shape=(nDaysMin, nFeatures)))
    model.add(LSTM(nNeurons, activation='relu'))
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')

    # fit model
    model.fit(XTrain, yTrain[:,0], epochs=1000, verbose=0)

    yPredListList = []
    for day in range(nTest):
        yPredListList.append([])
    XValidNew = XValid.copy()
    for day in range(k):
        yPred = model.predict(XValidNew, verbose=0)
        for i in range(len(yPred)):
            yPredListList[i].append(yPred[i][0])
        XValidNew = np.delete(XValidNew, 0, axis=1)
        yPred = np.expand_dims(yPred, 2)
        XValidNew = np.append(XValidNew, yPred, axis=1)

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError_kDay(yValid, yPredListList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(LSTM(nNeurons, activation='relu', return_sequences=True, input_shape=(nDaysMin, nFeatures)))
model.add(LSTM(nNeurons, activation='relu'))
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')

# fit model
model.fit(XTrain, yTrain[:,0], epochs=1000, verbose=0)

yPredListList = []
for day in range(nTest):
    yPredListList.append([])
XTestNew = XTest.copy()
for day in range(k):
    yPred = model.predict(XTestNew, verbose=0)
    for i in range(len(yPred)):
        yPredListList[i].append(yPred[i][0])
    XTestNew = np.delete(XTestNew, 0, axis=1)
    yPred = np.expand_dims(yPred, 2)
    XTestNew = np.append(XTestNew, yPred, axis=1)
    
MAPE = meanAbsolutePercentageError_kDay(yTest, yPredListList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy_kDay(yTest, yPredListList)
print('Test MdSA:', MdSA)

0 28.15282215708961
Updating best MAPE to 28.15282215708961...
Updating best seed to 0...
1 43.0278185364813
2 35.70799692070865
3 33.20350346154514
4 34.4943849797818
5 178.28289715784393
6 34.05724253389047
7 4.768249577445977
Updating best MAPE to 4.768249577445977...
Updating best seed to 7...
8 38.53663737045889
9 378.237327753451
10 36.67646762251382
11 0.7463787638973229
Updating best MAPE to 0.7463787638973229...
Updating best seed to 11...
12 30.953703475157504
13 99.83725143392438
14 1.2335154266434822
15 99.84969370399875
16 0.29345260274761165
Updating best MAPE to 0.29345260274761165...
Updating best seed to 16...
17 4.694755358443724
18 99.86099531737814
19 41.707212406386525
20 99.85075888932604
21 0.3622813821877656
22 41.66027673978932
23 0.13691705685238253
Updating best MAPE to 0.13691705685238253...
Updating best seed to 23...
24 42.109054828321156
25 1286.0499066807026
26 30.10382329555725
27 2.9410534238564203
28 39.65617487857317
29 36.70850494937876
30 1.5979487

# Bidirectional LSTM

In [9]:
from tensorflow.keras.layers import Bidirectional

# define model
nNeurons = 50
nFeatures = 1

bestValidMAPE = 100
bestSeed = -1
for seed in range(100):
    tf.random.set_seed(seed=seed)
    model = Sequential()
    model.add(Bidirectional(LSTM(nNeurons, activation='relu'), input_shape=(nDaysMin, nFeatures)))
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')

    # fit model
    model.fit(XTrain, yTrain[:,0], epochs=1000, verbose=0)

    yPredListList = []
    for day in range(nTest):
        yPredListList.append([])
    XValidNew = XValid.copy()
    for day in range(k):
        yPred = model.predict(XValidNew, verbose=0)
        for i in range(len(yPred)):
            yPredListList[i].append(yPred[i][0])
        XValidNew = np.delete(XValidNew, 0, axis=1)
        yPred = np.expand_dims(yPred, 2)
        XValidNew = np.append(XValidNew, yPred, axis=1)

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError_kDay(yValid, yPredListList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(Bidirectional(LSTM(nNeurons, activation='relu'), input_shape=(nDaysMin, nFeatures)))
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')

# fit model
model.fit(XTrain, yTrain[:,0], epochs=1000, verbose=0)

yPredListList = []
for day in range(nTest):
    yPredListList.append([])
XTestNew = XTest.copy()
for day in range(k):
    yPred = model.predict(XTestNew, verbose=0)
    for i in range(len(yPred)):
        yPredListList[i].append(yPred[i][0])
    XTestNew = np.delete(XTestNew, 0, axis=1)
    yPred = np.expand_dims(yPred, 2)
    XTestNew = np.append(XTestNew, yPred, axis=1)
    
MAPE = meanAbsolutePercentageError_kDay(yTest, yPredListList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy_kDay(yTest, yPredListList)
print('Test MdSA:', MdSA)

0 1.5035175259989129
Updating best MAPE to 1.5035175259989129...
Updating best seed to 0...
1 0.875677738915505
Updating best MAPE to 0.875677738915505...
Updating best seed to 1...
2 99.8407631942072
3 0.4167751047940387
Updating best MAPE to 0.4167751047940387...
Updating best seed to 3...
4 1.6368073287505012
5 8.081803767881334
6 0.15718462504303643
Updating best MAPE to 0.15718462504303643...
Updating best seed to 6...
7 0.3818830710905131
8 1.1357103141406173
9 0.2876190759175124
10 1.4053330169083622
11 0.4321780689875845
12 11.20526287237612
13 0.9309696612562993
14 4.3585364069893275
15 4.393760961309734
16 99.89896646216354
17 0.5506260809068939
18 2.1874969935929123
19 2.1771563740135926
20 99.84172983680767
21 99.88843034956903
22 0.8416689037583044
23 0.3571452704586209
24 2.973783851803712
25 15.856522795074536
26 99.8888812327687
27 5.320033884967132
28 0.6928008729976419
29 4.111914392990601
30 4.918494520429573
31 0.6187755047359076
32 3.798049574061646
33 3.8948729901

# CNN LSTM

In [10]:
# import tensorflow as tf
# from tensorflow.keras import Sequential
# from tensorflow.keras.layers import LSTM, Dense
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.layers import Flatten
# from tensorflow.compat.v1.keras.layers import TimeDistributed
# from tensorflow.keras.layers import Conv1D
# from tensorflow.keras.layers import MaxPooling1D

# nSeq = 5
# nSteps = 3
# nNeurons = 50
# nFeatures = 1
# nFilters = 64

# XTestListNew = XTestList.copy()
# print(XTestListNew.shape)

# model = Sequential()
# model.add(TimeDistributed(Conv1D(filters=nFilters, kernel_size=1, activation='relu'), input_shape=(None, nSteps, nFeatures)))
# model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
# model.add(TimeDistributed(Flatten()))
# model.add(LSTM(nNeurons, activation='relu'))
# model.add(Dense(1))
# opt = Adam(learning_rate=0.1)
# model.compile(optimizer=opt, loss='mse')

# XTestCNN = XTestListNew.reshape((XTestListNew.shape[0], nSeq, nSteps, nFeatures))
# yPred = model.predict(XTestCNN, verbose=0)

# XTestListNew = np.delete(XTestListNew, 0, axis=1)
# XTestListNew = np.append(XTestListNew, yPred, axis=1)

In [11]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten
from tensorflow.compat.v1.keras.layers import TimeDistributed
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D

# Number of subsequences to break X into (we do 15 = 5x3, 5 subsequences of size 3 each)
nSeq = 3
nSteps = 2

# define model
nNeurons = 50
nFeatures = 1
nFilters = 64

bestValidMAPE = 100
bestSeed = -1

# Reshape input
XTrainCNN = XTrainList.reshape((XTrainList.shape[0], nSeq, nSteps, nFeatures))

# print(XTrainCNN.shape)
# print(XValidCNN.shape)
# print(XTestCNN.shape)

for seed in range(100):
    tf.random.set_seed(seed=seed)
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=nFilters, kernel_size=1, activation='relu'), input_shape=(None, nSteps, nFeatures)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(nNeurons, activation='relu'))
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')

    # fit model
    model.fit(XTrainCNN, yTrain[:,0], epochs=1000, verbose=0)
    
    yPredListList = []
    for day in range(nTest):
        yPredListList.append([])
    XValidListNew = XValidList.copy()
    for day in range(k):
        XValidCNN = XValidListNew.reshape((XValidListNew.shape[0], nSeq, nSteps, nFeatures))
        yPred = model.predict(XValidCNN, verbose=0)
        for i in range(len(yPred)):
            yPredListList[i].append(yPred[i][0])
        XValidListNew = np.delete(XValidListNew, 0, axis=1)
        XValidListNew = np.append(XValidListNew, yPred, axis=1)

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError_kDay(yValid, yPredListList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=nFilters, kernel_size=1, activation='relu'), input_shape=(None, nSteps, nFeatures)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(nNeurons, activation='relu'))
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')
# fit model
model.fit(XTrainCNN, yTrain[:,0], epochs=1000, verbose=0)

yPredListList = []
for day in range(nTest):
    yPredListList.append([])
XTestListNew = XTestList.copy()
for day in range(k):
    XTestCNN = XTestListNew.reshape((XTestListNew.shape[0], nSeq, nSteps, nFeatures))
    yPred = model.predict(XTestCNN, verbose=0)
    for i in range(len(yPred)):
        yPredListList[i].append(yPred[i][0])
    XTestListNew = np.delete(XTestListNew, 0, axis=1)
    XTestListNew = np.append(XTestListNew, yPred, axis=1)
    
MAPE = meanAbsolutePercentageError_kDay(yTest, yPredListList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy_kDay(yTest, yPredListList)
print('Test MdSA:', MdSA)

0 2.5646811583171085
Updating best MAPE to 2.5646811583171085...
Updating best seed to 0...
1 38.76034177246358
2 2.4352571471979743
Updating best MAPE to 2.4352571471979743...
Updating best seed to 2...
3 33.26741497736624
4 3.798385752300516
5 38.43285346557805
6 33.48605434001216
7 7.30410170983985
8 0.9796388151419866
Updating best MAPE to 0.9796388151419866...
Updating best seed to 8...
9 45.176599393435325
10 38.101368682335135
11 1.1579550717927014
12 5.462657712123244
13 1.1630241648436492
14 4.380689049341165
15 1.8001221645262433
16 0.8264168371539664
Updating best MAPE to 0.8264168371539664...
Updating best seed to 16...
17 4.140182731231799
18 2.1820606690251405
19 1.561508659383829
20 1.2516193705761398
21 9.432038299607546
22 3.271547893962837
23 2.296460057021558
24 5.823258663966155
25 40.656512565628034
26 1.3740433285975624
27 35.61143943127403
28 0.6440618539765943
Updating best MAPE to 0.6440618539765943...
Updating best seed to 28...
29 3.5046132067986604
30 4.5673

# ConvLSTM

In [12]:
from tensorflow.keras.layers import ConvLSTM2D

# Number of subsequences to break X into (we do 15 = 5x3, 5 subsequences of size 3 each)
nSeq = 3
nSteps = 2
# Each input is rows x columns, we have rows=1 and columns=nSteps

# define model
nNeurons = 50
nFeatures = 1
nFilters = 64

bestValidMAPE = 100
bestSeed = -1

# Reshape input
XTrainConv = XTrainList.reshape((XTrainList.shape[0], nSeq, 1, nSteps, nFeatures))
# XValidConv = XValidList.reshape((XValidList.shape[0], nSeq, 1, nSteps, nFeatures))
# XTestConv = XTestList.reshape((XTestList.shape[0], nSeq, 1, nSteps, nFeatures))

for seed in range(100):
    tf.random.set_seed(seed=seed)
    model = Sequential()
    model.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', input_shape=(nSeq, 1, nSteps, nFeatures)))
    model.add(Flatten())
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')
    
    # fit model
    model.fit(XTrainConv, yTrain[:,0], epochs=1000, verbose=0)

    yPredListList = []
    for day in range(nTest):
        yPredListList.append([])
    XValidListNew = XValidList.copy()
    for day in range(k):
        XValidConv = XValidListNew.reshape((XValidListNew.shape[0], nSeq, 1, nSteps, nFeatures))
        yPred = model.predict(XValidConv, verbose=0)
        for i in range(len(yPred)):
            yPredListList[i].append(yPred[i][0])
        XValidListNew = np.delete(XValidListNew, 0, axis=1)
        XValidListNew = np.append(XValidListNew, yPred, axis=1)

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError_kDay(yValid, yPredListList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', input_shape=(nSeq, 1, nSteps, nFeatures)))
model.add(Flatten())
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')
# fit model
model.fit(XTrainConv, yTrain[:,0], epochs=1000, verbose=0)

yPredListList = []
for day in range(nTest):
    yPredListList.append([])
XTestListNew = XTestList.copy()
for day in range(k):
    XTestConv = XTestListNew.reshape((XTestListNew.shape[0], nSeq, 1, nSteps, nFeatures))
    yPred = model.predict(XTestConv, verbose=0)
    for i in range(len(yPred)):
        yPredListList[i].append(yPred[i][0])
    XTestListNew = np.delete(XTestListNew, 0, axis=1)
    XTestListNew = np.append(XTestListNew, yPred, axis=1)
    
MAPE = meanAbsolutePercentageError_kDay(yTest, yPredListList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy_kDay(yTest, yPredListList)
print('Test MdSA:', MdSA)

0 0.3088308109918116
Updating best MAPE to 0.3088308109918116...
Updating best seed to 0...
1 2226.876417408008
2 0.31612102355516536
3 99.95855228603321
4 2.720803835051445
5 1.0059424370970813
6 97.35933180457901
7 99.7557793980249
8 0.2540440438176329
Updating best MAPE to 0.2540440438176329...
Updating best seed to 8...
9 0.7347629698656859
10 0.8604442820333345
11 4.030483087329763
12 3.554752276127661
13 1.4351073628067943
14 0.32163965476809336
15 1.4831284977619248
16 2.8134188605129737
17 0.9763611558883147
18 1.3378716453436907
19 0.14105917408334998
Updating best MAPE to 0.14105917408334998...
Updating best seed to 19...
20 0.12295701537864864
Updating best MAPE to 0.12295701537864864...
Updating best seed to 20...
21 2.7522114569344165
22 0.6190863673621947
23 97.8833524547051
24 3.0461792889227546
25 0.17716024871752092
26 0.14338002912623266
27 2.4143803405630293
28 105.36595440421874
29 5.249937431872594
30 2.7562549547646045
31 0.9485766064710062
32 16.082712999245537
3

In [13]:
# import matplotlib.pyplot as plt
# from statsmodels.tsa.statespace.sarimax import SARIMAX

# # ARIMA
# def getPredictions(X, nDaysInFuture=5, invertible=False, pdqTuple = (1, 2, 2)):
#     p, d, q = pdqTuple
#     predList = []
#     Xcopy = X.copy()
#     for i in range(nDaysInFuture):
#         if invertible:
#             model = SARIMAX(Xcopy, order=(p, d, q))

#             model_fit = model.fit(disp=False)

#             # make prediction
#             yhat = model_fit.predict(len(Xcopy), len(Xcopy), typ='levels')
#         else:
#             model = SARIMAX(Xcopy, order=(p, d, q), enforce_invertibility=False)

#             model_fit = model.fit(disp=False)

#             # make prediction
#             yhat = model_fit.predict(len(Xcopy), len(Xcopy), typ='levels')
#         Xcopy = np.append(Xcopy, yhat)
#         predList.append(np.around(yhat[0]))
        
#     return predList

# yPredARIMA = getPredictions(nCasesGreaterDaysSinceList[-15-k:-k], nDaysInFuture=k)

# plt.figure(figsize=(10,10))
# datesForPlottingList = datesGreaterDaysSinceList[-k:]
# groundTruthList = nCasesGreaterDaysSinceList[-k:]

# if countryName == 'US':
#     plt.ylabel('Number of cases (in millions)', fontsize=20)
#     plt.plot(datesForPlottingList, np.divide(groundTruthList, 1e6), '-o', label='Ground Truth');
#     plt.plot(datesForPlottingList, np.divide(yPredVanillaLSTM[-1], 1e6), '-o', label='Vanilla LSTM');
#     plt.plot(datesForPlottingList, np.divide(yPredStackedLSTM[-1], 1e6), '-o', label='Stacked LSTM');
#     plt.plot(datesForPlottingList, np.divide(yPredBidirectionalLSTM[-1], 1e6), '-o', label='Bidirectional LSTM');
#     plt.plot(datesForPlottingList, np.divide(yPredCNNLSTM[-1], 1e6), '-o', label='CNN LSTM');
#     plt.plot(datesForPlottingList, np.divide(yPredConvLSTM[-1], 1e6), '-o', label='ConvLSTM');
#     plt.plot(datesForPlottingList, np.divide(yPredConvLSTM[-1], 1e6), '-o', label='ConvLSTM');
#     plt.plot(datesForPlottingList, np.divide(yPredARIMA, 1e6), '-o', label='ARIMA');
# else:
#     plt.ylabel('Number of cases', fontsize=20)
#     plt.plot(datesForPlottingList, groundTruthList, '-o', label='Ground Truth');
#     plt.plot(datesForPlottingList, yPredVanillaLSTM[-1], '-o', label='Vanilla LSTM');
#     plt.plot(datesForPlottingList, yPredStackedLSTM[-1], '-o', label='Stacked LSTM');
#     plt.plot(datesForPlottingList, yPredBidirectionalLSTM[-1], '-o', label='Bidirectional LSTM');
#     plt.plot(datesForPlottingList, yPredCNNLSTM[-1], '-o', label='CNN LSTM');
#     plt.plot(datesForPlottingList, yPredConvLSTM[-1], '-o', label='ConvLSTM');
#     plt.plot(datesForPlottingList, yPredConvLSTM[-1], '-o', label='ConvLSTM');
#     plt.plot(datesForPlottingList, yPredARIMA, '-o', label='ARIMA');
# plt.xlabel('Date', fontsize=20);